In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
#OBTENER DATOS

#Descomprimir
import tarfile
tar = tarfile.open("/content/drive/My Drive/Datasets/Cats&Dogs/fastai-datasets-cats-vs-dogs-2.tar", 'r:')
#tar.extractall("/content/drive/My Drive/Datasets/Cats&Dogs")
tar.extractall()
tar.close()

#Borrar archivo .tar
#os.remove('/content/drive/My Drive/datasets/fastai-datasets-cats-vs-dogs-2.tar')

#Comprobar las carpetas
os.listdir()

['.config',
 'train',
 'sample',
 'test1',
 'valid',
 'models',
 'drive',
 'sample_data']

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Convolution2D(128, (3, 3), strides=(1,1), input_shape=(64,64,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, (3,3), strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               3211520   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Definir Datagens
train_data_gen = ImageDataGenerator(
  rescale = 1./255, #Normalizacion para que los valores de las imagenes para que queden entre 0 y 1
  shear_range = 0.2, #Direccion de rotacion contrareloj
  zoom_range = 0.2, #Aplicacion de zoom aleatorio
  horizontal_flip = True # Va a invertir horizontalmente la imagen aleatoriamente
)

test_data_gen = ImageDataGenerator(
  rescale = 1./255 #Normalizacion para que los valores de las imagenes para que queden entre 0 y 1
)
#Armar datasets
training_set = train_data_gen.flow_from_directory(
  "train",
  target_size = (64, 64), #Valores de nuestras capas de entrada (64, 64, 3)
  batch_size = 256,
  class_mode = "binary"
)

test_set = test_data_gen.flow_from_directory("valid",
                                           target_size = (64,64),
                                           batch_size = 256,
                                           class_mode = "binary"
                                           )


#Entrenar
model.fit_generator(
  training_set,
  epochs = 25,
  validation_data = test_set,
)


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
90/90 [==============================] - 88s 980ms/step - loss: 0.6827 - accuracy: 0.5752 - val_loss: 0.6295 - val_accuracy: 0.6380
Epoch 2/25
90/90 [==============================] - 83s 924ms/step - loss: 0.6053 - accuracy: 0.6796 - val_loss: 0.6032 - val_accuracy: 0.6715
Epoch 3/25
90/90 [==============================] - 83s 921ms/step - loss: 0.5735 - accuracy: 0.7017 - val_loss: 0.5563 - val_accuracy: 0.7245
Epoch 4/25
90/90 [==============================] - 83s 925ms/step - loss: 0.5362 - accuracy: 0.7311 - val_loss: 0.5431 - val_accuracy: 0.7255
Epoch 5/25
90/90 [==============================] - 84s 929ms/step - loss: 0.5164 - accuracy: 0.7457 - val_loss: 0.5104 - val_accuracy: 0.7500
Epoch 6/25
90/90 [==============================] - 84s 931ms/step - loss: 0.5045 - accuracy: 0.7515 - val_loss: 0.4914 - va

In [ ]:
#Predecir
from google.colab.files import upload #Colab
import numpy as np
#upload()
from keras.preprocessing import image
imagen = image.load_img("/content/gdrive/My Drive/Colab Notebooks/SIS421/CNN/p.jpg", target_size=(64,64))
imagen = image.img_to_array(imagen)
imagen = np.expand_dims(imagen, axis=0) #(64, 64, 3) -> (1, 64, 64, 3) 
res = model.predict_classes(imagen)
print('Resultado: ', res)
print('Etiquetas: ',training_set.class_indices)

#os.mkdir('/content/gdrive/My Drive/Modelos')
model.save("/content/gdrive/My Drive/Modelos/DogVSCatModel.h5")

from tensorflow.keras.models import load_model
model = load_model('/content/gdrive/My Drive/Modelos/DogVSCatModel.h5')


Resultado:  [[1]]
Etiquetas:  {'cats': 0, 'dogs': 1}
